###**Q1**.  Use [Lucidchart](https://sjsu.edu/it/services/applications/lucidchart.php) to create an Entity Relationship Diagram (ERD) for the following tables representing a customer order tracking system:

Tables and fields:
1. product:
    - product_id: INTEGER (Primary key)
    - name: TEXT not null
    - price: REAL


2. customer:
    - customer_id: INTEGER (Primary key)
    - name: TEXT not null
    - email: TEXT not null


3. purchase_order:
    - order_id: INTEGER (Primary key)
    - customer_id: INTEGER (Foreign key)
    - date: TEXT not null ("YYYY-MM-DD")


4. order_item:
    - order_id: INTEGER (Foreign key)
    - product_id: INTEGER (Foreign key)
    - quantity: INTEGER
    


Export the ERD as PDF and submit it in Canvas.

###**Q2**. Create SQLite tables and load data
1. Here are the csv files for the data for the four tables:
   - product: https://raw.githubusercontent.com/csbfx/cs133/main/product.csv
   - customer: https://raw.githubusercontent.com/csbfx/cs133/main/customer.csv
   - order_item: https://raw.githubusercontent.com/csbfx/cs133/main/order_item.csv
   - purchase_order: https://raw.githubusercontent.com/csbfx/cs133/main/purchase_order.csv
2. In this notebook, create the database and save it in a file called `store.db`, and create the four tables as described above.
3. Load the data in the csv files into the corresponding table.
4. Commit so that the data loaded to the tables to officially written to the tables.
5. Execute a query SELECT * from each table to make sure the data are properly loaded.
6. Execute a query using SELECT statement that queries with JOIN tables to find the purchase date, the products and quantities that a particular customer has purchased.

In [3]:
# 2.2 Create the database and save it in a file called store.db, and create the four tables as described above.
# Your code here . . .
import sqlite3
import pandas as pd

conn = sqlite3.connect('store.db')
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS product (
    product_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    price REAL
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS customer (
    customer_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    email TEXT NOT NULL
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS purchase_order (
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    date TEXT NOT NULL,
    FOREIGN KEY (customer_id) REFERENCES customer (customer_id)
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS order_item (
    order_id INTEGER,
    product_id INTEGER,
    quantity INTEGER,
    FOREIGN KEY (order_id) REFERENCES purchase_order (order_id),
    FOREIGN KEY (product_id) REFERENCES product (product_id),
    PRIMARY KEY (order_id, product_id)  -- composite primary key
);
''')

conn.commit()

In [4]:
# 2.3 Load the data in the csv files into the corresponding table.
# Your code here . . .
product = pd.read_csv("https://raw.githubusercontent.com/csbfx/cs133/main/product.csv")
customer = pd.read_csv("https://raw.githubusercontent.com/csbfx/cs133/main/customer.csv")
order_item = pd.read_csv("https://raw.githubusercontent.com/csbfx/cs133/main/order_item.csv")
purchase_order = pd.read_csv("https://raw.githubusercontent.com/csbfx/cs133/main/purchase_order.csv")

product.to_sql('product', conn, if_exists='append', index=False)
customer.to_sql('customer', conn, if_exists='append', index=False)
order_item.to_sql('order_item', conn, if_exists='append', index=False)
purchase_order.to_sql("purchase_order", conn, if_exists='append', index=False)

3

In [5]:
# 2.4 Commit so that the data loaded to the tables to officially written to the tables.
# Your code here . . .
conn.commit()

In [7]:
# 2.5 Execute a query SELECT * from each table to make sure the data are properly loaded.
# Your code here . . .
cursor.execute("SELECT * FROM product")
product_data = cursor.fetchall()
print(product_data)

cursor.execute("SELECT * FROM customer")
customer_data = cursor.fetchall()
print(customer_data)

cursor.execute("SELECT * FROM purchase_order")
purchase_order_data = cursor.fetchall()
print(purchase_order_data)

cursor.execute("SELECT * FROM order_item")
order_item_data = cursor.fetchall()
print(order_item_data)

[(0, 'bicycle', 400.0), (1, 'helmet', 45.0), (2, 'gloves', 23.0), (3, 'chain', 48.0)]
[(0, 'Wendy Lee', 'wlee@bike.com'), (1, 'Jason Brown', 'jb@speed.com'), (2, 'Harry Potter', 'hp@hogwarts.edu'), (3, 'Godric Gryffindor', 'gg@hogwards.edu')]
[(0, 0, '2020-10-19'), (1, 0, '2020-10-20'), (2, 1, '2020-10-20')]
[(0, 0, 1), (0, 1, 1), (1, 2, 2), (1, 3, 1)]


In [12]:
# 2.6 Execute a query using SELECT statement that queries with JOIN tables to find the purchase date,
#    the products and quantities that a particular customer has purchased.
# Query 1: Orders for Wendy Lee
# Query 2: Orders for Jason Brown
# Your code here . . .
query1 = '''
SELECT
    po.date AS purchase_date,
    p.name AS product_name,
    oi.quantity AS quantity
FROM
    purchase_order po
JOIN
    order_item oi ON po.order_id = oi.order_id
JOIN
    product p ON oi.product_id = p.product_id
WHERE
    po.customer_id = 0;
'''

query2 = '''
SELECT
    po.date AS purchase_date,
    p.name AS product_name,
    oi.quantity AS quantity
FROM
    purchase_order po
JOIN
    order_item oi ON po.order_id = oi.order_id
JOIN
    product p ON oi.product_id = p.product_id
WHERE
    po.customer_id = 1;
'''
cursor.execute(query1)
query1data = cursor.fetchall()
print(query1data)

cursor.execute(query2)
query2data = cursor.fetchall()
print(query2data)

[('2020-10-19', 'bicycle', 1), ('2020-10-19', 'helmet', 1), ('2020-10-20', 'gloves', 2), ('2020-10-20', 'chain', 1)]
[]


In [ ]:
# Additional tasks
# Appending a new row to one of the tables
# Deleting row(s) with "XXX"